# Background Removal Demo using OpenVINO and DeGirum PySDK

This demo notebook shows image segmentation and removing/adding background with U^2-Net and OpenVINO™.


## Requirements

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

import degirum as dg
import degirum_tools as dgtools

model_name = 'vision_background_removal--512x512_float_openvino_cpu_1'
zoo_url = 'https://cs.degirum.com/degirum/openvino_demos'
img_path = 'coco_hollywood.jpg'

zoo = dg.connect(dg.CLOUD, zoo_url, dgtools.get_token())
model = zoo.load_model(model_name)
img_rgb = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

## Run inference on an image

In [ ]:
# Preprocess
h_in, w_in, c_in = img_rgb.shape

# Inference
input = cv2.resize(img_rgb, (512,512)).astype(np.float32) / 255
input = input.transpose((2, 0, 1))[np.newaxis, ...]
res = model(input)
res = res.results[0]['data']

# Post-process
mask = cv2.resize(np.squeeze(res), (w_in, h_in))[..., np.newaxis]
masked_image = (img_rgb * mask + 255 * np.ones((h_in, w_in, c_in)) * (1 - mask)).astype(np.uint8)

# Show results
fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(1, 2, 1)
plt.title("Input", fontsize=20)
ax1.axis("off")
ax2 = fig.add_subplot(1, 2, 2)
plt.title("Masked", fontsize=20)
ax2.axis("off")
ax1.imshow(img_rgb)
ax2.imshow(masked_image)
plt.show()